In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
%matplotlib inline 

from hts.hierarchy import HierarchyTree

In [8]:
sales = pd.read_csv('/Users/user/hts-forecast/volume/data/raw/sales_train_evaluation.csv')
cal = pd.read_csv('/Users/user/hts-forecast/volume/data/raw/calendar.csv')

In [9]:
sales.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1932,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,4,0,0,0,0,3,3,0,1
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,1,2,1,1,0,0,0,0,0
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,0,2,0,0,0,2,3,0,1
3,HOBBIES_1_004_CA_1_evaluation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,1,0,4,0,1,3,0,2,6
4,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,2,1,0,0,2,1,0


In [10]:
#take the dataframe and only present the relevant category
def filter_aggregate_sales(frame, column_to_filter, column_value_to_filter, column_to_aggregate):
    frame = frame[frame[column_to_filter] == column_value_to_filter]
    frame = frame.groupby(column_to_aggregate).sum().T
    return frame

In [11]:
states = filter_aggregate_sales(frame=sales, column_to_filter='state_id', column_value_to_filter='CA',
                                 column_to_aggregate='state_id')

In [12]:
for state in sales.state_id.unique():
    states[state] = filter_aggregate_sales(frame=sales, column_to_filter='state_id', column_value_to_filter=state,
                                                    column_to_aggregate='state_id')

for store in sales.store_id.unique():
    states[store] = filter_aggregate_sales(frame=sales, column_to_filter='store_id', column_value_to_filter=store,
                                                    column_to_aggregate='store_id')    

for category in sales.cat_id.unique():
    states[category] = filter_aggregate_sales(frame=sales, column_to_filter='cat_id', column_value_to_filter=category,
                                                       column_to_aggregate='cat_id') 
for dept in sales.dept_id.unique():
    states[dept] = filter_aggregate_sales(frame=sales, column_to_filter='dept_id', column_value_to_filter=dept,
                                                       column_to_aggregate='dept_id') 

states['total'] = states.CA + states.TX + states.WI


states.head()

state_id,CA,TX,WI,CA_1,CA_2,CA_3,CA_4,TX_1,TX_2,TX_3,...,HOUSEHOLD,FOODS,HOBBIES_1,HOBBIES_2,HOUSEHOLD_1,HOUSEHOLD_2,FOODS_1,FOODS_2,FOODS_3,total
d_1,14195,9438,8998,4337,3494,4739,1625,2556,3852,3030,...,5689,23178,3610,154,4105,1584,2343,4094,16741,32631
d_2,13805,9630,8314,4155,3046,4827,1777,2687,3937,3006,...,5634,22758,3172,185,3858,1776,2216,4209,16333,31749
d_3,10108,6778,6897,2816,2121,3785,1386,1822,2731,2225,...,3927,17174,2497,185,2827,1100,1657,3174,12343,23783
d_4,11047,7381,6984,3051,2324,4232,1440,2258,2954,2169,...,3865,18878,2531,138,2732,1133,1508,3606,13764,25412
d_5,9925,5912,3309,2630,1942,3817,1536,1694,2492,1726,...,2729,14603,1714,100,1802,927,1209,2869,10525,19146


In [15]:
state = sales.state_id.unique()
store = sales.store_id.unique()
dept = sales.dept_id.unique()
cat = sales.cat_id.unique()


array(['HOBBIES', 'HOUSEHOLD', 'FOODS'], dtype=object)

In [34]:
total = {'total': list(state)}
state_h = {k: [v for v in store if v.startswith(k)] for k in state}
store_h = {k: [v for v in cat if v.startswith(k)] for k in store}
dept_h = {k: [v for v in dept if v.startswith(k)] for k in cat}

In [35]:
hierarchy = {**total, **state_h, **store_h, **dept_h}
hierarchy

{'total': ['CA', 'TX', 'WI'],
 'CA': ['CA_1', 'CA_2', 'CA_3', 'CA_4'],
 'TX': ['TX_1', 'TX_2', 'TX_3'],
 'WI': ['WI_1', 'WI_2', 'WI_3'],
 'CA_1': [],
 'CA_2': [],
 'CA_3': [],
 'CA_4': [],
 'TX_1': [],
 'TX_2': [],
 'TX_3': [],
 'WI_1': [],
 'WI_2': [],
 'WI_3': [],
 'HOBBIES': ['HOBBIES_1', 'HOBBIES_2'],
 'HOUSEHOLD': ['HOUSEHOLD_1', 'HOUSEHOLD_2'],
 'FOODS': ['FOODS_1', 'FOODS_2', 'FOODS_3']}

In [37]:
ht = HierarchyTree.from_nodes(nodes=hierarchy, df=states)

In [38]:
ht.children

[- CA
    |- CA_1
    |- CA_2
    |- CA_3
    - CA_4,
 - TX
    |- TX_1
    |- TX_2
    - TX_3,
 - WI
    |- WI_1
    |- WI_2
    - WI_3]

In [39]:
ht.children[0].children[2]

- CA_3